Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [17]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice
from copy import deepcopy

from tqdm.auto import tqdm
import numpy as np


from collections import namedtuple, defaultdict
from itertools import combinations
from random import choice, random
from copy import deepcopy

In [18]:
State = namedtuple('State', ['x', 'o'])

In [19]:
MAGIC = [2, 7, 6, 9, 5, 1, 4, 3, 8]

In [20]:
def print_board(pos):
    """Nicely prints the board"""
    for r in range(3):
        for c in range(3):
            i = r * 3 + c
            if MAGIC[i] in pos.x:
                print('X', end='')
            elif MAGIC[i] in pos.o:
                print('O', end='')
            else:
                print('.', end='')
        print()
    print()

In [21]:
def win(elements):
    """Checks is elements is winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))

def state_value(pos: State):
    """Evaluate state: +1 first player wins"""
    if win(pos.x):
        return 1
    elif win(pos.o):
        return -1
    else:
        return 0
    
    

In [6]:
def random_game():
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    while available:
        x = choice(list(available))
        state.x.add(x)
        trajectory.append(deepcopy(state))
        available.remove(x)
        if win(state.x) or not available:
            break

        o = choice(list(available))
        state.o.add(o)
        trajectory.append(deepcopy(state))
        available.remove(o)
        if win(state.o):
            break
    return trajectory

In [7]:
value_dictionary = defaultdict(float)
hit_state = defaultdict(int)
epsilon = 0.001

for steps in tqdm(range(500_000)):
    trajectory = random_game()
    final_reward = state_value(trajectory[-1])
    for state in trajectory:
        hashable_state = (frozenset(state.x), frozenset(state.o))
        hit_state[hashable_state] += 1
        value_dictionary[hashable_state] = value_dictionary[
            hashable_state
        ] + epsilon * (final_reward - value_dictionary[hashable_state])

100%|██████████| 500000/500000 [01:34<00:00, 5295.82it/s]


In [8]:
sorted(value_dictionary.items(), key=lambda e: e[1], reverse=True)[:10]

[((frozenset({1, 4, 5, 6, 7}), frozenset({2, 3, 8, 9})), 0.9163606208461627),
 ((frozenset({1, 3, 6, 8, 9}), frozenset({2, 4, 5, 7})), 0.9156884847308895),
 ((frozenset({1, 2, 4, 5, 8}), frozenset({3, 6, 7, 9})), 0.913206393246716),
 ((frozenset({2, 3, 5, 8, 9}), frozenset({1, 4, 6, 7})), 0.9131195127594756),
 ((frozenset({1, 2, 4, 6, 7}), frozenset({3, 5, 8, 9})), 0.9122459102842707),
 ((frozenset({2, 4, 5, 7, 8}), frozenset({1, 3, 6, 9})), 0.9117175379847842),
 ((frozenset({4, 5, 6, 7, 9}), frozenset({1, 2, 3, 8})), 0.9116291671519361),
 ((frozenset({1, 4, 5, 7, 9}), frozenset({2, 3, 6, 8})), 0.9110080894969314),
 ((frozenset({1, 5, 7, 8, 9}), frozenset({2, 3, 4, 6})), 0.9109190085054368),
 ((frozenset({3, 4, 6, 8, 9}), frozenset({1, 2, 5, 7})), 0.9108298383437806)]

In [9]:
len(hit_state)

5477

one agent randome and the other q-dict

In [36]:
def epsilon_greedy(epsilon, state, value_dict):
    """Epsilon-greedy exploration strategy"""
    hashable_state = (frozenset(state.x), frozenset(state.o))
    available_actions = list(set(range(1, 9+1)) - set(state.x) - set(state.o))

    if not available_actions or random() < epsilon:
        # Explore: choose a random action
        if available_actions:
            return choice(available_actions)
        else:
            return choice(list(range(1, 9+1)))  # Choose any action if none are available
    else:
        # Exploit: choose the action with the highest Q-value
        q_values = {action: value_dict[hashable_state + (action,)] for action in available_actions}
        return max(q_values, key=q_values.get)



In [51]:
def q_learning_train(value_dict, hit_state, epsilon, alpha, gamma, num_episodes):
    """Q-learning training"""
    for episode in tqdm(range(num_episodes)):
        state = State(set(), set())
        available = set(range(1, 9+1))
        while available:
            action = epsilon_greedy(epsilon, state, value_dict)
            state_prime = deepcopy(state)
            state_prime.x.add(action)
            if win(state_prime.x) or not available:
                reward = state_value(state_prime)
                update_q_value(state, action, reward, state_prime, value_dict, alpha, gamma)
                break

            o = choice(list(available))
            state_prime.o.add(o)
            if win(state_prime.o):
                reward = state_value(state_prime)
                update_q_value(state, action, reward, state_prime, value_dict, alpha, gamma)
                break

            update_q_value(state, action, 0, state_prime, value_dict, alpha, gamma)
            state = deepcopy(state_prime)

            # Ensure that action is present in the available set before removing it
            if action in available:
                available.remove(action)

        # # Print the Q-values periodically to observe the learning progress
        # if episode % 1000 == 0:
        #     print(f"Episode: {episode}, Q-Values: {sorted(value_dict.items(), key=lambda e: e[1], reverse=True)[:5]}")

In [52]:
def update_q_value(state, action, reward, state_prime, value_dict, alpha, gamma):
    """Update Q-value based on the Q-learning update rule"""
    hashable_state = (frozenset(state.x), frozenset(state.o))
    hashable_state_prime = (frozenset(state_prime.x), frozenset(state_prime.o))
    
    available_actions_prime = list(set(range(1, 9+1)) - set(state_prime.x) - set(state_prime.o))
    
    if available_actions_prime:
        max_q_value_prime = max(value_dict[hashable_state_prime + (a,)] for a in available_actions_prime)
    else:
        max_q_value_prime = 0  # If no valid actions available, set the max Q-value to 0
    
    value_dict[hashable_state + (action,)] += alpha * (
            reward + gamma * max_q_value_prime -
            value_dict[hashable_state + (action,)])


In [53]:
# Q-learning hyperparameters
alpha = 0.1  # learning rate
gamma = 0.9  # discount factor
num_episodes = 500000

In [54]:
# Initialize Q-values and train the agent
value_dictionary = defaultdict(float)
hit_state = defaultdict(int)
epsilon = 0.1  # exploration-exploitation trade-off
q_learning_train(value_dictionary, hit_state, epsilon, alpha, gamma, num_episodes)

# After the training loop, print the final Q-values
print("Final Q-Values:")
print(sorted(value_dictionary.items(), key=lambda e: e[1], reverse=True)[:10])

  0%|          | 0/500000 [00:00<?, ?it/s]

100%|██████████| 500000/500000 [01:35<00:00, 5247.75it/s]

Final Q-Values:
[((frozenset({8, 1, 2, 4}), frozenset({9, 3, 5}), 6), 0.9999999999999996), ((frozenset({1, 2, 4}), frozenset({8, 3, 6}), 9), 0.9999999999999996), ((frozenset({8, 1, 2, 3}), frozenset({9, 5, 6}), 4), 0.9999999999999996), ((frozenset({1, 2, 4}), frozenset({1, 3, 7}), 9), 0.9999999999999996), ((frozenset({1, 2, 4}), frozenset({3, 5, 6}), 9), 0.9999999999999996), ((frozenset({1, 2, 4}), frozenset({8, 3, 7}), 9), 0.9999999999999996), ((frozenset({1, 2, 4}), frozenset({8, 5, 7}), 9), 0.9999999999999996), ((frozenset({1, 2, 4, 5}), frozenset({8, 9, 3}), 6), 0.9999999999999996), ((frozenset({1, 2, 4}), frozenset({8, 5, 6}), 9), 0.9999999999999996), ((frozenset({1, 2, 4}), frozenset({1, 3, 5}), 9), 0.9999999999999996)]


In [59]:
def test_agent(value_dict):
    state = State(set(), set())
    available = set(range(1, 9+1))
    while available:
        print_board(state)
        action = epsilon_greedy(0, state, value_dict)  # Choose greedy action (no exploration)
        state.x.add(action)
        if win(state.x) or not available:
            print("Agent wins!")
            break

        o = choice(list(available))
        state.o.add(o)
        if win(state.o):
            print("Opponent wins!")
            break

        # Ensure that action is present in the available set before removing it
        if action in available:
            available.remove(action)

    # Print the final board state
    print("Final Board State:")
    print_board(state)

In [64]:
test_agent(value_dictionary)

...
...
...

...
..X
..O

...
.XX
..O

Agent wins!
Final Board State:
...
XXX
..O



both players learn and use q-dict:

In [68]:
def epsilon_greedy(epsilon, state, value_dict):
    """Epsilon-greedy exploration strategy"""
    hashable_state = (frozenset(state.x), frozenset(state.o))
    available_actions = list(set(range(1, 9+1)) - set(state.x) - set(state.o))

    if not available_actions or random() < epsilon:
        # Explore: choose a random action
        if available_actions:
            return choice(available_actions)
        else:
            return choice(list(range(1, 9+1)))  # Choose any action if none are available
    else:
        # Exploit: choose the action with the highest Q-value
        q_values = {action: value_dict[hashable_state + (action,)] for action in available_actions}
        return max(q_values, key=q_values.get)

In [69]:
def update_q_value(state, action, reward, state_prime, value_dict, alpha, gamma):
    """Update Q-value based on the Q-learning update rule"""
    hashable_state = (frozenset(state.x), frozenset(state.o))
    hashable_state_prime = (frozenset(state_prime.x), frozenset(state_prime.o))
    
    available_actions_prime = list(set(range(1, 9+1)) - set(state_prime.x) - set(state_prime.o))
    
    if available_actions_prime:
        max_q_value_prime = max(value_dict[hashable_state_prime + (a,)] for a in available_actions_prime)
    else:
        max_q_value_prime = 0  # If no valid actions available, set the max Q-value to 0
    
    value_dict[hashable_state + (action,)] += alpha * (
            reward + gamma * max_q_value_prime -
            value_dict[hashable_state + (action,)])

In [70]:
def q_learning_train(agent_value_dict, opponent_value_dict, epsilon, alpha, gamma, num_episodes):
    """Q-learning training for both agent and opponent"""
    for episode in tqdm(range(num_episodes)):
        agent_state = State(set(), set())
        opponent_state = State(set(), set())
        agent_available = set(range(1, 9+1))
        opponent_available = set(range(1, 9+1))
        
        while agent_available:
            # Agent's turn
            agent_action = epsilon_greedy(epsilon, agent_state, agent_value_dict)
            agent_state_prime = deepcopy(agent_state)
            agent_state_prime.x.add(agent_action)
            
            if win(agent_state_prime.x) or not agent_available:
                reward = state_value(agent_state_prime)
                update_q_value(agent_state, agent_action, reward, agent_state_prime, agent_value_dict, alpha, gamma)
                break

            # Opponent's turn
            opponent_action = epsilon_greedy(epsilon, opponent_state, opponent_value_dict)
            opponent_state_prime = deepcopy(opponent_state)
            opponent_state_prime.o.add(opponent_action)

            if win(opponent_state_prime.o) or not opponent_available:
                reward = state_value(opponent_state_prime)
                update_q_value(opponent_state, opponent_action, reward, opponent_state_prime, opponent_value_dict, alpha, gamma)
                break

            update_q_value(agent_state, agent_action, 0, agent_state_prime, agent_value_dict, alpha, gamma)
            agent_state = deepcopy(agent_state_prime)
            agent_available.remove(agent_action)

            update_q_value(opponent_state, opponent_action, 0, opponent_state_prime, opponent_value_dict, alpha, gamma)
            opponent_state = deepcopy(opponent_state_prime)
            opponent_available.remove(opponent_action)

In [71]:
# Initialize Q-values for both agent and opponent
agent_value_dictionary = defaultdict(float)
opponent_value_dictionary = defaultdict(float)
agent_hit_state = defaultdict(int)
opponent_hit_state = defaultdict(int)

# Placeholder for other initializations if you have any

# Q-learning hyperparameters
alpha = 0.1  # learning rate
gamma = 0.9  # discount factor
num_episodes = 500000

# Exploration-exploitation trade-off
epsilon = 0.1  

In [72]:
# Train both agent and opponent
q_learning_train(agent_value_dictionary, opponent_value_dictionary, epsilon, alpha, gamma, num_episodes)

# After the training loop, print the final Q-values for both agent and opponent
print("Final Q-Values for Agent:")
print(sorted(agent_value_dictionary.items(), key=lambda e: e[1], reverse=True)[:10])

print("Final Q-Values for Opponent:")
print(sorted(opponent_value_dictionary.items(), key=lambda e: e[1], reverse=True)[:10])

100%|██████████| 500000/500000 [02:12<00:00, 3760.49it/s]


Final Q-Values for Agent:
[((frozenset({1, 2, 4}), frozenset(), 9), 0.9999999999999996), ((frozenset({1, 2, 3, 4}), frozenset(), 9), 0.9999999999999996), ((frozenset({1, 2, 3, 4, 5}), frozenset(), 8), 0.9999999999999996), ((frozenset({1, 6}), frozenset(), 8), 0.9999999999999996), ((frozenset({1, 2, 6}), frozenset(), 7), 0.9999999999999996), ((frozenset({1, 2, 6}), frozenset(), 8), 0.9999999999999996), ((frozenset({1, 2, 3, 4, 6}), frozenset(), 8), 0.9999999999999996), ((frozenset({1, 2, 3, 7}), frozenset(), 6), 0.9999999999999996), ((frozenset({1, 2, 3, 4, 7}), frozenset(), 8), 0.9999999999999996), ((frozenset({1, 2, 3, 5}), frozenset(), 7), 0.9999999999999996)]
Final Q-Values for Opponent:
[((frozenset(), frozenset(), 1), 0.0), ((frozenset(), frozenset(), 2), 0.0), ((frozenset(), frozenset(), 3), 0.0), ((frozenset(), frozenset(), 4), 0.0), ((frozenset(), frozenset(), 5), 0.0), ((frozenset(), frozenset(), 6), 0.0), ((frozenset(), frozenset(), 7), 0.0), ((frozenset(), frozenset(), 8), 0

In [74]:
# ... (previous code)

# Test the learned agent against a random opponent
def test_agent_vs_opponent(agent_value_dict, opponent_value_dict):
    agent_state = State(set(), set())
    opponent_state = State(set(), set())
    game_state = State(set(), set())  # Shared game state
    agent_available = set(range(1, 9+1))
    opponent_available = set(range(1, 9+1))
    
    while agent_available:
        print("Agent's turn:")
        print_board(game_state)
        agent_action = epsilon_greedy(0, game_state, agent_value_dict)  # Choose greedy action (no exploration)
        game_state.x.add(agent_action)
        
        if win(game_state.x) or not agent_available:
            print("Agent wins!")
            break

        print("Opponent's turn:")
        print_board(game_state)
        opponent_action = epsilon_greedy(0, game_state, opponent_value_dict)  # Choose greedy action (no exploration)
        game_state.o.add(opponent_action)

        if win(game_state.o) or not opponent_available:
            print("Opponent wins!")
            break

        # Ensure that action is present in the available set before removing it
        if agent_action in agent_available:
            agent_available.remove(agent_action)

        if opponent_action in opponent_available:
            opponent_available.remove(opponent_action)

    # Print the final board state
    print("Final Board State:")
    print_board(game_state)

# Test the learned agent against a random opponent
test_agent_vs_opponent(agent_value_dictionary, opponent_value_dictionary)


Agent's turn:
...
...
...

Opponent's turn:
...
..X
...

Agent's turn:
O..
..X
...

Opponent's turn:
O..
..X
.X.

Agent's turn:
O..
..X
OX.

Opponent's turn:
O..
.XX
OX.

Agent's turn:
O..
.XX
OXO

Agent wins!
Final Board State:
O..
XXX
OXO

